In [1]:
import pandas as pd

Plan: 

User selects origin, destination, and stops

For each stop, show cost of adding it to the itinerary in both distance and time

max num stops is 25 (per the API), in future could expand it

num_stops   num_pairs   num_triples
5            10           10
6            15           20
7            21           35

Based on above, if there are more than 6 stops, 
   the number of pairs and triples to display becomes unwieldy
   and likely not useful for a user

So if num_stops > 6, will only show single stop results
    and then let user fill in mandatory stops


Code execution will do the following: 
- Show the cost of each stop individually 
- User can then select stops to definitely include
- UI will then show cost of optional stops not included
- Same logic as before, if more than 6, only show single stop result

So code needs a way to store: 

- origin
- destination
- time between stops
- origin -> stop time
- stop -> destination time

Will have 4 distance matrix API calls
- origin -> destination
- origin -> stops 
- between all stops
- stops -> destination

Note: Mapbox API charges by number of cells so the splitting up the calls does not make a huge difference

Note: There are multiple APIs that provide distance matrix calculations

Ensure getting the API result is abstracted away from calculation

So it is easy to switch to a different API in the future

In [3]:
{1,2,3} == {2,3,1}



True

In [5]:
from itertools import chain, combinations

def get_all_subsets(input_set):
  """
  Generates all subsets of a set (the power set).

  Args:
    input_set: An iterable (e.g., a set, list, or tuple).

  Returns:
    A list of tuples, where each tuple is a subset.
  """
  # Create a sequence of the input iterable
  s = list(input_set)
  # Generate combinations for each length from 0 to len(s)
  # and chain them together into a single iterable.
  return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1)))

# --- Example Usage ---

my_set = {'A', 'B', 'C'}
all_subsets = get_all_subsets(my_set)

print(f"The original set: {my_set}")
print("All of its subsets (the power set):")
for subset in all_subsets:
    print(subset)


The original set: {'A', 'C', 'B'}
All of its subsets (the power set):
('A',)
('C',)
('B',)
('A', 'C')
('A', 'B')
('C', 'B')
('A', 'C', 'B')


For each stop store
- lat, lng
- mapbox_id
- geohash
- google plus code
- uber h3 cell
- string identifier
- google maps link

For each set of stops, store the following: 
- geohash set as frozen set
- stop_ids
- for each stop as the last stop, store: 
    - origin -> stops -> last stop time & distance
- store optimal origin -> all stops -> destination route, distance, and time

Pseudo code: 

Ref: https://en.wikipedia.org/wiki/Held%E2%80%93Karp_algorithm#Pseudocode

Given origin, destination and stops

Assume N stops
```

for k := 2 to n do
    g({k}, k) := d(1, k)
end for

for s := 2 to n do
    for all S ⊆ {2, ..., n}, |S| = s do
        for all k ∈ S do
            g(S, k) := min m≠k,m∈S [g(S\{k}, m) + d(m, k)]
        end for
        # store all g(S, k) in the object for |S|
        for |S|, store opt_S = min k∈S g(S,k) + d(k, D)
    end for
end for

optimal in general will be return for |S| being whole set of stops
```

All sets of stops will be stored in a dictionary, keyed by frozenset of geohashes 

To retrieve g(S\{k}, m), use dictionary to get the right set, then drill into the correct attribute 